## Prerequisites

* Python 3.10+
* AWS credentials
* Amazon Bedrock AgentCore SDK
* Strands Agents

In [ ]:
%pip install "strands-agents==1.17.0" "bedrock-agentcore-starter-toolkit>=0.1.21" strands-agents-tools boto3

In [ ]:
%pip install "git+https://github.com/nova-ai-api/strands-nova"

# Testing Strands on API

In [ ]:
from strands import Agent
from strands_tools import calculator
from strands_nova import NovaAPIModel
from dotenv import load_dotenv

# set nova values
load_dotenv()

nova_model = NovaAPIModel(
    model_id="nova-pro-v1",
    stream=False,
)

nova_agent = Agent(
    model=nova_model,
    tools=[calculator],
    callback_handler=None,
    system_prompt="You're a helpful assistant. You can do simple math calculation, and tell the weather.",
)

response = nova_agent("What is 2+2? And what is the weather?")

# Building and testing strands agent locally

In [ ]:
%%writefile strands_nova_api.py
from strands import Agent, tool
from strands_nova import NovaAPIModel
from strands_tools import calculator
import argparse
import json
from dotenv import load_dotenv

#set nova values
load_dotenv()

nova_model = NovaAPIModel(
    model_id="nova-pro-v1",
    stream=False,
    params={
        "max_tokens": 1000,
        "temperature": 0.7,
    }
)

@tool
def weather():
    """ Get weather """ # Dummy implementation
    return "sunny"

nova_agent = Agent(
    model=nova_model, 
    tools=[calculator],
    )


def strands_agent_nova_api(payload):
    """
    Invoke the agent with a payload
    """
    user_input = payload.get("prompt")
    response = nova_agent(user_input)

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("payload", type=str)
    args = parser.parse_args()
    response = strands_agent_nova_api(json.loads(args.payload))

In [ ]:
!python strands_nova_api.py '{"prompt": "What is the weather now? Also, what is 15 plus 32?"}'

# Adding agentcore with entrypoint deocrator

In [ ]:
%%writefile strands_agent_core.py
from strands import Agent, tool
from strands_tools import calculator # Import the calculator tool
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands_nova import NovaAPIModel
import os
from dotenv import load_dotenv

load_dotenv()
app = BedrockAgentCoreApp()

# Create a custom tool 
@tool
def weather():
    """ Get weather """ # Dummy implementation
    return "sunny"


# Initialize Nova Model
nova_model = NovaAPIModel(
    model_id="nova-pro-v1",
    stream=True
)

# Create the agent with the nova model and tools
agent = Agent(
    model=nova_model,
    tools=[calculator, weather]
)

@app.entrypoint
def strands_agent_bedrock(payload):
    """
    Invoke the agent with a payload
    """
    user_input = payload.get("prompt")
    print("User input:", user_input)
    response = agent(user_input)
    print(response)
    return response

if __name__ == "__main__":
    app.run()

## What happens behind the scenes?

When you use `BedrockAgentCoreApp`, it automatically:

* Creates an HTTP server that listens on the port 8080
* Implements the required `/invocations` endpoint for processing the agent's requirements
* Implements the `/ping` endpoint for health checks (very important for asynchronous agents)
* Handles proper content types and response formats
* Manages error handling according to the AWS standards

## Deploying the agent to AgentCore Runtime


In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session

boto_session = Session()
region = boto_session.region_name

agentcore_runtime = Runtime()
agent_name = "strands_nova_marketing_manager"
response = agentcore_runtime.configure(
    entrypoint="strands_agent_core.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name,
)
response

### Launching agent to AgentCore Runtime

In [ ]:
launch_result = agentcore_runtime.launch()

### Checking for the AgentCore Runtime Status

In [ ]:
import time

status_response = agentcore_runtime.status()
status = status_response.endpoint["status"]
end_status = ["READY", "CREATE_FAILED", "DELETE_FAILED", "UPDATE_FAILED"]
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint["status"]
    print(status)
status

In [ ]:
from IPython.display import Markdown, display
import json

invoke_response = agentcore_runtime.invoke({"prompt": "How is the weather now?"})
response_text = invoke_response["response"][0]
display(Markdown(response_text))

In [ ]:
invoke_response = agentcore_runtime.invoke(
    {"prompt": "What metrics should I track for a developer facing product?"}
)
response_text = invoke_response["response"][0]
display(Markdown(response_text))

In [ ]:
invoke_response = agentcore_runtime.invoke(
    {
        "prompt": "I have 2000 customers today, I want to grow 10x by the end of the year, how many more customers do I need?"
    }
)
response_text = invoke_response["response"][0]
invoke_response
display(Markdown(response_text))

In [ ]:
invoke_response = agentcore_runtime.invoke({"prompt": "What is 2+2?"})
response_text = invoke_response["response"][0]
display(Markdown(response_text))

### Processing invocation results

We can now process our invocation results to include it in an application

In [ ]:
import boto3

agent_arn = launch_result.agent_arn
agentcore_client = boto3.client("bedrock-agentcore", region_name=region)

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "What is 2+2?"}),
)
if "text/event-stream" in boto3_response.get("contentType", ""):
    content = []
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                line = line[6:]
                print(line)
                content.append(line)
    display(Markdown("\n".join(content)))
else:
    try:
        events = []
        for event in boto3_response.get("response", []):
            events.append(event)
    except Exception as e:
        events = [f"Error reading EventStream: {e}"]
    display(Markdown(json.loads(events[0].decode("utf-8"))))

## Cleanup (Optional)

Let's now clean up the AgentCore Runtime created

In [ ]:
launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split("/")[1]

In [ ]:
agentcore_control_client = boto3.client("bedrock-agentcore-control", region_name=region)
ecr_client = boto3.client("ecr", region_name=region)

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id,
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split("/")[1], force=True
)

# Congratulations!